In [1]:
from ipywidgets import interact, interact_manual, widgets
import numpy as np
from numpy.random import randn
import gzip

In [2]:
def afficher(v_in,nb_bytes_per_lines=128):
    v_in_len=v_in.shape[0]
    nb_lines=int(v_in_len/nb_bytes_per_lines)+1
   
    for j in range(nb_lines):
        s_line='%.3d:'%(j)
        for i in range(nb_bytes_per_lines):
            k=j*nb_bytes_per_lines+i
            if k<v_in_len:
                s_line='%s%.2x'%(s_line,v_in[k])
        print(s_line)

def comp_gzip(v_in):
    v_out = np.frombuffer(gzip.compress(v_in),dtype=np.uint8)
    return v_out


   
def generate_rnd_bits(freq_ech=32.0,nb_of_osc=1,nb_bits=4096):
    v_list_nb_premiers=[3,7,11,13,17,19,23,29,31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    v_nb_elements=[]
    for i in range(nb_of_osc):
        v_nb_elements.append(v_list_nb_premiers[i])
  ### durations are in us
    mean_delay=2.5e-4
    std_dev_mean_delay=0 ## sigma_mu_delay

    ## sigma_delay
    std_dev_delay=1e-6
    std_dev_std_dev_delay=0 ## sigma_sigma_delay

    nb_osc=len(v_nb_elements)

    v_half_periods_mean=np.zeros((nb_osc,),dtype=np.float64)
    v_half_periods_std=np.zeros((nb_osc,),dtype=np.float64)
    for i in range(nb_osc):
        v_half_periods_mean[i]=(mean_delay+std_dev_mean_delay*np.random.randn(v_nb_elements[i])).sum()
        v_half_periods_std[i]=np.sqrt(((std_dev_delay+std_dev_std_dev_delay*np.random.randn(v_nb_elements[i]))**2).sum())

    v_raw_bits=np.zeros((nb_bits,),dtype=np.uint8)
    v_raw_bits_tmp=np.zeros((nb_bits,),dtype=np.uint8)
    T_ech=1.0/freq_ech
    p=10
    for j in range(nb_osc):
        M=int(1.5*(T_ech*nb_bits)/(v_half_periods_mean[i]+p*v_half_periods_std[i]))
        v_phase=(v_half_periods_mean[i]+v_half_periods_std[i]*np.random.randn(M)).cumsum()+np.random.rand()*v_half_periods_mean[i]
        ros_state_init=np.random.rand()>0.5
        v_raw_bits_tmp=np.array(((np.where(np.diff(np.array(v_phase/T_ech,dtype=np.uint64))==1)[0][:nb_bits])%2)^ros_state_init,dtype=np.uint8)
        v_raw_bits^=(v_raw_bits_tmp)
    nb_bytes=int(nb_bits/8)
    v_hex_in = np.zeros((nb_bytes,),dtype=np.uint8)
    for i in range(8):
        v_hex_in+=v_raw_bits[i:nb_bytes*8:8]*(2**i)
    return v_hex_in
   
def generate_rnd_bits_gui(freq_ech=32.0,nb_of_osc=1,nb_bits=4096,nb_bytes_per_lines=80):
    v_list_nb_premiers=[3,7,11,13,17,19,23,29,31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    v_nb_elements=[]
    for i in range(nb_of_osc):
        v_nb_elements.append(v_list_nb_premiers[i])
  ### durations are in us
    mean_delay=2.5e-4
    std_dev_mean_delay=0 ## sigma_mu_delay

    ## sigma_delay
    std_dev_delay=1e-6
    std_dev_std_dev_delay=0 ## sigma_sigma_delay

    nb_osc=len(v_nb_elements)

    v_half_periods_mean=np.zeros((nb_osc,),dtype=np.float64)
    v_half_periods_std=np.zeros((nb_osc,),dtype=np.float64)
    for i in range(nb_osc):
        v_half_periods_mean[i]=(mean_delay+std_dev_mean_delay*np.random.randn(v_nb_elements[i])).sum()
        v_half_periods_std[i]=np.sqrt(((std_dev_delay+std_dev_std_dev_delay*np.random.randn(v_nb_elements[i]))**2).sum())

    v_raw_bits=np.zeros((nb_bits,),dtype=np.uint8)
    v_raw_bits_tmp=np.zeros((nb_bits,),dtype=np.uint8)
    T_ech=1.0/freq_ech
    p=10
    for j in range(nb_osc):
        M=int(1.5*(T_ech*nb_bits)/(v_half_periods_mean[i]+p*v_half_periods_std[i]))
        v_phase=(v_half_periods_mean[i]+v_half_periods_std[i]*np.random.randn(M)).cumsum()+np.random.rand()*v_half_periods_mean[i]
        ros_state_init=np.random.rand()>0.5
        v_raw_bits_tmp=np.array(((np.where(np.diff(np.array(v_phase/T_ech,dtype=np.uint64))==1)[0][:nb_bits])%2)^ros_state_init,dtype=np.uint8)
        v_raw_bits^=(v_raw_bits_tmp)
    nb_bytes=int(nb_bits/8)
    v_hex_in = np.zeros((nb_bytes,),dtype=np.uint8)
    for i in range(8):
        v_hex_in+=v_raw_bits[i:nb_bytes*8:8]*(2**i)
    print('\nFICHIER NON COMPRESSE : %d BYTES'%(v_hex_in.shape[0]))
    afficher(v_hex_in,nb_bytes_per_lines)
    v_comp = np.frombuffer(gzip.compress(v_hex_in),dtype=np.uint8)
    print('\nFICHIER COMPRESSE : %d BYTES'%(v_comp.shape[0]))
    afficher(v_comp,nb_bytes_per_lines)

In [6]:
im = interact(generate_rnd_bits_gui)
display(im)

im.widget.children[0].min=1
im.widget.children[0].max=32
im.widget.children[0].description='Frequence d echantillonnage en MHz :'
im.widget.children[0].style={'description_width': '30%'}
im.widget.children[0].layout=widgets.Layout(width='50%')

im.widget.children[1].min=1
im.widget.children[1].max=23
im.widget.children[1].description='Nombre d oscillateurs à inverseurs :'
im.widget.children[1].style={'description_width': '30%'}
im.widget.children[1].layout=widgets.Layout(width='50%')


im.widget.children[2].min=1024
im.widget.children[2].max=int(1024*16)
im.widget.children[2].step=8
im.widget.children[2].value=4096
im.widget.children[2].description='Nombre de bits a generer:'
im.widget.children[2].style={'description_width': '30%'}
im.widget.children[2].layout=widgets.Layout(width='50%')

im.widget.children[3].min=4
im.widget.children[3].max=512
im.widget.children[3].step=1
im.widget.children[3].value=16
im.widget.children[3].description='Nombre de bits a afficher par ligne:'
im.widget.children[3].style={'description_width': '30%'}
im.widget.children[3].layout=widgets.Layout(width='50%')

interactive(children=(FloatSlider(value=32.0, description='freq_ech', max=96.0, min=-32.0), IntSlider(value=1,…

<function __main__.generate_rnd_bits_gui>